<a href="https://colab.research.google.com/github/RickyF404/Tesi/blob/main/Anomaly_autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import gdown

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer

In [2]:
url_no_temp = "https://drive.google.com/uc?id=1jerpsOqjogEnnriBkHOrDDEeh1wJry_o"
output_no_temp = "data_no_temperature.csv"
gdown.download(url_no_temp, output_no_temp)

url_temp = " https://drive.google.com/uc?id=1RofjUHZS_UAnbF6Xe74jTp9z4Mmk7swH"
output_temp = "data_temperature.csv"
gdown.download(url_temp, output_temp)

Downloading...
From: https://drive.google.com/uc?id=1jerpsOqjogEnnriBkHOrDDEeh1wJry_o
To: /content/data_no_temperature.csv
100%|██████████| 1.44M/1.44M [00:00<00:00, 16.8MB/s]
Downloading...
From:  https://drive.google.com/uc?id=1RofjUHZS_UAnbF6Xe74jTp9z4Mmk7swH
To: /content/data_temperature.csv
100%|██████████| 1.66M/1.66M [00:00<00:00, 33.9MB/s]


'data_temperature.csv'

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

df_original = pd.read_csv("/content/data_no_temperature.csv", encoding = "utf-8")
df_original["timestamp"] = pd.to_datetime(df_original["timestamp"])
df_original["timestamp"] = df_original["timestamp"].dt.tz_localize(None)
df_original = df_original.sort_values("timestamp")

end = "2025-01-01"
df = df_original[(df_original["timestamp"] < end)]


url_anomaly = "https://drive.google.com/uc?id=1onDmjfJio6nVRaaQSOgd6wW_XyuyyCra"
output_anomaly = "data_anomaly.csv"
gdown.download(url_anomaly, output_anomaly)

df_anomaly = pd.read_csv("/content/data_anomaly.csv", encoding = "utf-8")

mapping = {
    "F1": "Mode 1",
    "F2": "Mode 2",
    "F3": "Mode 3",
    "F4": "Mode 4",
    "F5": "Mode 5",
    "F6": "Mode 6",
    "Temperature": "temperatura"
}

df_anomaly = df_anomaly.rename(columns=mapping)

Downloading...
From: https://drive.google.com/uc?id=1onDmjfJio6nVRaaQSOgd6wW_XyuyyCra
To: /content/data_anomaly.csv
100%|██████████| 990k/990k [00:00<00:00, 12.4MB/s]


# AutoEncoder senza temperatura

In [ ]:
def build_ae(input_dim, latent_dim=2):
  #encoder
  input_enc = layers.Input(shape=(input_dim,))
  x = layers.Dense(64, activation="relu")(input_enc)
  x = layers.Dense(32, activation="relu")(x)
  x = layers.Dense(16, activation="relu")(x)
  z = layers.Dense(latent_dim, activation="relu", name="latent")(x)

  #decoder
  x = layers.Dense(16, activation="relu")(z)
  x = layers.Dense(32, activation="relu")(x)
  x = layers.Dense(64, activation="relu")(x)
  output_dec = layers.Dense(input_dim, activation="linear")(x)

  #autoencoder
  ae = keras.Model(input_enc, output_dec, name="autoencoder")
  ae.compile(optimizer="adam", loss="mse")
  return ae

Scenario di danno DS1, 1 mese di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_60)






df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS1, 2 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_60)






df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS1, 4 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_60)






df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS1, 6 mesi di dati

In [ ]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_20)







df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_40)






df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_60)






df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_80)






df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS2, 1 mese di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_60)






df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS2, 2 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_60)






df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS2, 4 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_60)






df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS2, 6 mesi di dati

In [ ]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_20)







df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_40)






df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_60)






df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_80)






df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS3, 1 mese di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_60)






df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS3, 2 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_60)






df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS3, 4 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_60)






df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS3, 6 mesi di dati

In [ ]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_20)







df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_40)






df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_60)






df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_80)






df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS4, 1 mese di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_60)






df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS4, 2 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_60)






df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS4, 4 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_60)






df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS4, 6 mesi di dati

In [ ]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_20)







df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_40)






df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_60)






df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_80)






df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])

  ae = build_ae(input_dim=X_train.shape[1], latent_dim=2)

  ae.load_weights(f"/content/drive/MyDrive/AE_no_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse


  X_pred_test = ae.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val - X_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

# Autoencoder con temperatura

In [4]:
df_temp = pd.read_csv("/content/data_temperature.csv", encoding = "utf-8")
df_temp["timestamp"] = pd.to_datetime(df_temp["timestamp"])
df_temp["timestamp"] = df_temp["timestamp"].dt.tz_localize(None)
df_temp = df_temp.sort_values("timestamp")

end = "2025-01-01"
df = df_temp[(df_temp["timestamp"] < end)]


def build_ae_temp(input_dim = 7, output_dim = 6, latent_dim=2):
  #encoder
  input_enc = layers.Input(shape=(input_dim,))
  x = layers.Dense(48, activation="relu")(input_enc)
  x = layers.Dense(24, activation="relu")(x)
  z = layers.Dense(latent_dim, activation="relu", name="latent")(x)

  #decoder
  x = layers.Dense(24, activation="relu")(z)
  x = layers.Dense(48, activation="relu")(x)
  output_dec = layers.Dense(output_dim, activation="linear")(x)

  #autoencoder
  ae = keras.Model(input_enc, output_dec, name="autoencoder")
  ae.compile(optimizer="adam", loss="mse")
  return ae

Scenario di danno DS1, 1 mese di dati

In [5]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_20)






df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_60)





df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_80)




df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS1_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS1, 2 mesi di dati

In [6]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_20)






df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_60)





df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_80)




df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS1_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS1, 4 mesi di dati

In [7]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_20)






df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_60)





df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_80)




df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS1_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS1, 6 mesi di dati

In [8]:
df_DS1_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_20)






df_DS1_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_40)





df_DS1_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_60)





df_DS1_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_80)




df_DS1_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS1") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS1_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS1_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS1_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS2, 1 mese di dati

In [9]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_20)






df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_60)





df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_80)




df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS2_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS2, 2 mesi di dati

In [10]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_20)






df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_60)





df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_80)




df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS2_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS2, 4 mesi di dati

In [11]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_20)






df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_60)





df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_80)




df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS2_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS2, 6 mesi di dati

In [12]:
df_DS2_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_20)






df_DS2_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_40)





df_DS2_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_60)





df_DS2_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_80)




df_DS2_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS2") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS2_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS2_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS2_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS3, 1 mese di dati

In [13]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_20)






df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_80)




df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS3_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS3, 2 mesi di dati

In [14]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_20)






df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_80)




df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS3_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS3, 4 mesi di dati

In [15]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_20)






df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_80)




df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS3_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS3, 6 mesi di dati

In [16]:
df_DS3_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_20)






df_DS3_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_40)





df_DS3_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_60)





df_DS3_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_80)




df_DS3_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS3") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS3_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS3_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS3_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     

Scenario di danno DS4, 1 mese di dati

In [17]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_20)






df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_80)




df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 1
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_1month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_1m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_1m_anomaly_DS4_100)


    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           20           1.0
1     [2024-02]            1           20           1.0
2     [2024-03]            1           20           1.0
3     [2024-04]            1           20           1.0
4     [2024-05]            1           20           1.0
5     [2024-06]            1           20           1.0
6     [2024-07]            1           20           1.0
7     [2024-08]            1           20           1.0
8     [2024-09]            1           20           1.0
9     [2024-10]            1           20           1.0
10    [2024-11]            1           20           1.0
11    [2024-12]            1           20           1.0

    train_months  window_size  damage size  anomaly rate
0     [2024-01]            1           40           1.0
1     [2024-02]            1           40           1.0
2     [2024-03]            1           40           1.0
3     [2024-04]            1           40   

Scenario di danno DS4, 2 mesi di dati

In [18]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_20)






df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_80)




df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 2
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_2month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_2m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_2m_anomaly_DS4_100)


           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           20           1.0
1   [2024-02, 2024-03]            2           20           1.0
2   [2024-03, 2024-04]            2           20           1.0
3   [2024-04, 2024-05]            2           20           1.0
4   [2024-05, 2024-06]            2           20           1.0
5   [2024-06, 2024-07]            2           20           1.0
6   [2024-07, 2024-08]            2           20           1.0
7   [2024-08, 2024-09]            2           20           1.0
8   [2024-09, 2024-10]            2           20           1.0
9   [2024-10, 2024-11]            2           20           1.0
10  [2024-11, 2024-12]            2           20           1.0

           train_months  window_size  damage size  anomaly rate
0   [2024-01, 2024-02]            2           40           1.0
1   [2024-02, 2024-03]            2           40           1.0
2   [2024-03, 2024-04]            2           40   

Scenario di danno DS4, 4 mesi di dati

In [19]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_20)






df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_80)




df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 4
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_4month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_4m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_4m_anomaly_DS4_100)


                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4           20   
1  [2024-02, 2024-03, 2024-04, 2024-05]            4           20   
2  [2024-03, 2024-04, 2024-05, 2024-06]            4           20   
3  [2024-04, 2024-05, 2024-06, 2024-07]            4           20   
4  [2024-05, 2024-06, 2024-07, 2024-08]            4           20   
5  [2024-06, 2024-07, 2024-08, 2024-09]            4           20   
6  [2024-07, 2024-08, 2024-09, 2024-10]            4           20   
7  [2024-08, 2024-09, 2024-10, 2024-11]            4           20   
8  [2024-09, 2024-10, 2024-11, 2024-12]            4           20   

   anomaly rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  
6           1.0  
7           1.0  
8           1.0  

                            train_months  window_size  damage size  \
0  [2024-01, 2024-02, 2024-03, 2024-04]            4    

Scenario di danno DS4, 6 mesi di dati

In [20]:
df_DS4_20 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 20)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_20[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":20, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_20 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_20)






df_DS4_40 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 40)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_40[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":40, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_40 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_40)





df_DS4_60 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 60)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_60[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":60, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_60 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_60)





df_DS4_80 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 80)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_80[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":80, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_80 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_80)




df_DS4_100 = df_anomaly[(df_anomaly["Damage Scenario"] == "DS4") & (df_anomaly["Damage Size"] == 100)]

#parametri
features = ["Mode 1", "Mode 2", "Mode 3", "Mode 4", "Mode 5", "Mode 6", "temperatura"]
results = []
window_size = 6
n_months = 12
start_date = df["timestamp"].min()

for m in range(1, n_months - window_size + 2):
  train_start = start_date + pd.DateOffset(months=m-1)
  train_end = start_date + pd.DateOffset(months=m-1+window_size)
  train_data = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]

  scaler_X = StandardScaler()

  X_train = scaler_X.fit_transform(train_data[features])
  X_val = scaler_X.transform(df_DS4_100[features])


  Y_train = X_train[:, :6]

  ae_temp = build_ae_temp(input_dim=X_train.shape[1], output_dim=Y_train.shape[1] , latent_dim=2)

  ae_temp.load_weights(f"/content/drive/MyDrive/AE_temp_6month_{m}.weights.h5")


#calcolo della soglia su dati normali
  X_pred_train = ae_temp.predict(X_train, verbose=0)
  mse_train = np.mean(np.square(X_train[:, :6] - X_pred_train), axis=1)
  mean_mse = np.mean(mse_train)
  std_mse = np.std(mse_train)
  threshold = mean_mse + 3 * std_mse

  Y_pred_test = ae_temp.predict(X_val, verbose=0)
  mse_test = np.mean(np.square(X_val[:, :6] - Y_pred_test), axis=1)

  anomalies = mse_test > threshold
  anomaly_rate = np.mean(anomalies)

  train_months = sorted(train_data["timestamp"].dt.to_period("M").unique())
  results.append({"train_months": [str(x) for x in train_months], "window_size": len(train_months), "damage size":100, "anomaly rate": anomaly_rate})

results_df_6m_anomaly_DS4_100 = pd.DataFrame(results)
print("\n", results_df_6m_anomaly_DS4_100)


                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...            6   
2  [2024-03, 2024-04, 2024-05, 2024-06, 2024-07, ...            6   
3  [2024-04, 2024-05, 2024-06, 2024-07, 2024-08, ...            6   
4  [2024-05, 2024-06, 2024-07, 2024-08, 2024-09, ...            6   
5  [2024-06, 2024-07, 2024-08, 2024-09, 2024-10, ...            6   
6  [2024-07, 2024-08, 2024-09, 2024-10, 2024-11, ...            6   

   damage size  anomaly rate  
0           20           1.0  
1           20           1.0  
2           20           1.0  
3           20           1.0  
4           20           1.0  
5           20           1.0  
6           20           1.0  

                                         train_months  window_size  \
0  [2024-01, 2024-02, 2024-03, 2024-04, 2024-05, ...            6   
1  [2024-02, 2024-03, 2024-04, 2024-05, 2024-06, ...     